In [1]:
import pulp
import pandas as pd
from collections import defaultdict
import math

In [2]:
# ----------------------------
# 1) READ DATA FROM FILES
# ----------------------------
DAYS = ["Mon", "Tue", "Wed", "Thu", "Fri"]

# 🔹 Load subteams and bays from CSV instead of hardcoding
subteams_df = pd.read_csv("subteams.csv").set_index("Subteam")
bays_df = pd.read_csv("bays.csv").set_index("Bay")

subteams = list(subteams_df.index)
bays = list(bays_df.index)

total_seats = int(bays_df['Capacity'].sum())
total_people = int(subteams_df['Members'].sum())
days_required = 3

print(f"Total bay capacity (sum of all bays): {total_seats}")
print(f"Total members across subteams: {total_people}")
print(f"Weekly seat-days available: {total_seats * len(DAYS)}; required seats per week: {total_people * days_required}")

Total bay capacity (sum of all bays): 1113
Total members across subteams: 1671
Weekly seat-days available: 5565; required seats per week: 5013


In [3]:
print("Subteams:")
display(subteams_df)
print("Bays:")
display(bays_df)

Subteams:


,Team,Shift-timing,Members
Subteam,,,
A1,A,09:00-18:00,27
A2,A,09:00-18:01,21
B1,B,09:00-18:02,26
B2,B,09:00-18:03,23
B3,B,09:00-18:04,2
B4,B,13:00-22:00,19
C1,C,15:00-00:00,63
C2,C,09:00-18:00,400
C3,C,09:00-18:00,84


Bays:


,Team,Capacity
Bay,,
Seat_A1,A,25
Seat_A2,A,3
Seat_B1,B,34
Seat_B2,B,6
Seat_C1,C,55
Seat_C2,C,126
Seat_C3,C,11
Seat_C4,C,82
Seat_C5,C,115


In [4]:
# ----------------------------
# 2) MILP MODEL
# ----------------------------
model = pulp.LpProblem("Seat_Allocation_SoftAttendance", pulp.LpMinimize)

# Variables
x = pulp.LpVariable.dicts("x", (subteams, DAYS), cat="Binary")           # subteam s present (full) on day d
assign = pulp.LpVariable.dicts("assign", (subteams, bays, DAYS), lowBound=0, cat="Integer")  # number of people
y = pulp.LpVariable.dicts("y", (subteams, bays, DAYS), cat="Binary")     # indicates bay used by subteam that day
splits = pulp.LpVariable.dicts("splits", (subteams, DAYS), lowBound=0, cat="Integer")        # extra bays beyond 1
z = pulp.LpVariable.dicts("z", (subteams, bays, DAYS), cat="Binary")     # z=1 if entire subteam sits in one bay that day

attend_short = pulp.LpVariable.dicts("attend_short", subteams, lowBound=0, cat="Integer")

# Constraints
for s in subteams:
    model += pulp.lpSum(x[s][d] for d in DAYS) + attend_short[s] >= days_required

for s in subteams:
    M_s = int(subteams_df.loc[s, "Members"])
    for d in DAYS:
        model += pulp.lpSum(assign[s][b][d] for b in bays) == M_s * x[s][d]

for b in bays:
    cap_b = int(bays_df.loc[b, "Capacity"])
    for d in DAYS:
        model += pulp.lpSum(assign[s][b][d] for s in subteams) <= cap_b

for s in subteams:
    M_s = int(subteams_df.loc[s, "Members"])
    for b in bays:
        cap_b = int(bays_df.loc[b, "Capacity"])
        for d in DAYS:
            model += assign[s][b][d] <= cap_b * y[s][b][d]
            model += assign[s][b][d] >= 1 * y[s][b][d]
            model += assign[s][b][d] <= M_s * x[s][d]

for s in subteams:
    for d in DAYS:
        model += splits[s][d] >= pulp.lpSum(y[s][b][d] for b in bays) - 1

for s in subteams:
    M_s = int(subteams_df.loc[s, "Members"])
    for b in bays:
        for d in DAYS:
            model += assign[s][b][d] >= M_s * z[s][b][d]
            model += z[s][b][d] <= y[s][b][d]

for s in subteams:
    model += pulp.lpSum(x[s][d] for d in DAYS) >= 1  

for s in subteams:
    model += attend_short[s] <= days_required

In [5]:
# ----------------------------
# 3) OBJECTIVE
# ----------------------------
split_penalty = 1.0
attend_penalty = 1000.0  
model += (
    pulp.lpSum(y[s][b][d] for s in subteams for b in bays for d in DAYS)
    + split_penalty * pulp.lpSum(splits[s][d] for s in subteams for d in DAYS)
    + attend_penalty * pulp.lpSum(attend_short[s] for s in subteams)
)

In [6]:
# ----------------------------
# 4) SOLVE MILP
# ----------------------------
solver = pulp.PULP_CBC_CMD(msg=False, timeLimit=120)
status = model.solve(solver)
status_str = pulp.LpStatus[status]
print("MILP solver status:", status_str)


MILP solver status: Optimal


In [7]:
# ----------------------------
# 5) EXTRACT SOLUTION (if Optimal) or FALLBACK
# ----------------------------
def build_solution_from_model():
    schedule_rows = []
    expanded_rows = []
    for s in subteams:
        M_s = int(subteams_df.loc[s, "Members"])
        present_days = [d for d in DAYS if pulp.value(x[s][d]) >= 0.5]
        day_map = {}
        for d in present_days:
            bays_assigned = []
            for b in bays:
                val = int(round(pulp.value(assign[s][b][d]) or 0))
                if val > 0:
                    bays_assigned.append({"Bay": b, "Assigned": val})
                    expanded_rows.append({
                        "Subteam": s,
                        "Team": subteams_df.loc[s,"Team"],
                        "Day": d,
                        "Shift": subteams_df.loc[s,"Shift-timing"],
                        "Assigned_Bay": b,
                        "Assigned_count": val
                    })
            day_map[d] = bays_assigned

        schedule_rows.append({
            "Subteam": s,
            "Team": subteams_df.loc[s,"Team"],
            "Members": M_s,
            "Shift": subteams_df.loc[s,"Shift-timing"],
            "Days_present": ", ".join(present_days),
            "Assignments_by_day": day_map,
            "Attend_short": int(round(pulp.value(attend_short[s] or 0)))
        })

    schedule_df = pd.DataFrame(schedule_rows).set_index("Subteam")
    expanded_df = pd.DataFrame(expanded_rows)
    return schedule_df, expanded_df

if status_str == "Optimal":
    schedule_df, expanded_df = build_solution_from_model()
    print("\nMILP produced an optimal feasible solution.")
    display(schedule_df)
    display(expanded_df)
else:
    print("MILP did not produce an optimal solution. Falling back to Greedy constructive scheduler...")

    # ----------------------------
    # Greedy fallback (largest-first) -- ENSURES at least 3 days per subteam (no slack here)
    # ----------------------------
    def greedy_scheduler_force3(sub_df, bays_df, DAYS):
        rem_cap = {(b,d): int(bays_df.loc[b,"Capacity"]) for b in bays for d in DAYS}
        assign_res = defaultdict(int)  
        x_res = {(s,d): 0 for s in subteams for d in DAYS}

        order = sorted(subteams, key=lambda s: int(sub_df.loc[s,"Members"]), reverse=True)

        for s in order:
            M_s = int(sub_df.loc[s,"Members"])

            day_caps = { d: sum(rem_cap[(b,d)] for b in bays) for d in DAYS }
            feasible_days = [d for d in DAYS if day_caps[d] >= M_s]
            if not feasible_days:
                full_day = max(day_caps.items(), key=lambda kv: kv[1])[0]
            else:
                full_day = max(feasible_days, key=lambda d: day_caps[d])

            needed = M_s
            for b in sorted(bays, key=lambda b: rem_cap[(b,full_day)], reverse=True):
                if needed <= 0:
                    break
                take = min(rem_cap[(b,full_day)], needed)
                if take > 0:
                    assign_res[(s,b,full_day)] += take
                    rem_cap[(b,full_day)] -= take
                    needed -= take
            if needed > 0:
                raise RuntimeError(f"Greedy couldn't place full meeting for {s} on {full_day}")

            x_res[(s,full_day)] = 1

            days_to_assign = days_required - 1
            for _ in range(days_to_assign):
                day_caps = { d: sum(rem_cap[(b,d)] for b in bays) for d in DAYS }
                candidate_days = sorted(DAYS, key=lambda d: (-day_caps[d], d))
                for best_day in candidate_days:
                    if best_day == full_day and len(candidate_days) > 1:
                        continue
                    needed = M_s
                    for b in sorted(bays, key=lambda b: rem_cap[(b,best_day)], reverse=True):
                        if needed <= 0:
                            break
                        take = min(rem_cap[(b,best_day)], needed)
                        if take > 0:
                            assign_res[(s,b,best_day)] += take
                            rem_cap[(b,best_day)] -= take
                            needed -= take
                    if needed == 0:
                        x_res[(s,best_day)] = 1
                        break
                else:
                    raise RuntimeError(f"Greedy couldn't place day for {s} (need additional day).")

        # build dfs
        expanded_rows = []
        schedule_rows = []
        for s in subteams:
            M_s = int(sub_df.loc[s,"Members"])
            present_days = [d for d in DAYS if x_res[(s,d)] == 1]
            day_map = {}
            for d in present_days:
                bays_assigned = []
                for b in bays:
                    val = assign_res[(s,b,d)]
                    if val > 0:
                        bays_assigned.append({"Bay": b, "Assigned": val})
                        expanded_rows.append({
                            "Subteam": s,
                            "Team": sub_df.loc[s,"Team"],
                            "Day": d,
                            "Shift": sub_df.loc[s,"Shift-timing"],
                            "Assigned_Bay": b,
                            "Assigned_count": val
                        })
                day_map[d] = bays_assigned
            schedule_rows.append({
                "Subteam": s,
                "Team": sub_df.loc[s,"Team"],
                "Members": M_s,
                "Shift": sub_df.loc[s,"Shift-timing"],
                "Days_present": ", ".join(present_days),
                "Assignments_by_day": day_map
            })

        schedule_df = pd.DataFrame(schedule_rows).set_index("Subteam")
        expanded_df = pd.DataFrame(expanded_rows)
        return schedule_df, expanded_df


MILP produced an optimal feasible solution.


,Team,Members,Shift,Days_present,Assignments_by_day,Attend_short
Subteam,,,,,,
A1,A,27,09:00-18:00,"Tue, Wed, Thu","{'Tue': [{'Bay': 'Seat_C1', 'Assigned': 27}], ...",0
A2,A,21,09:00-18:01,"Mon, Wed, Fri","{'Mon': [{'Bay': 'Seat_A1', 'Assigned': 21}], ...",0
B1,B,26,09:00-18:02,"Mon, Tue, Wed","{'Mon': [{'Bay': 'Seat_D1', 'Assigned': 26}], ...",0
B2,B,23,09:00-18:03,"Tue, Thu, Fri","{'Tue': [{'Bay': 'Seat_A1', 'Assigned': 23}], ...",0
B3,B,2,09:00-18:04,"Tue, Wed, Thu","{'Tue': [{'Bay': 'Seat_A1', 'Assigned': 2}], '...",0
B4,B,19,13:00-22:00,"Tue, Wed, Thu","{'Tue': [{'Bay': 'Seat_B1', 'Assigned': 19}], ...",0
C1,C,63,15:00-00:00,"Wed, Thu, Fri","{'Wed': [{'Bay': 'Seat_C4', 'Assigned': 63}], ...",0
C2,C,400,09:00-18:00,"Mon, Thu, Fri","{'Mon': [{'Bay': 'Seat_B1', 'Assigned': 34}, {...",0
C3,C,84,09:00-18:00,"Wed, Thu, Fri","{'Wed': [{'Bay': 'Seat_E3', 'Assigned': 84}], ...",0


,Subteam,Team,Day,Shift,Assigned_Bay,Assigned_count
0,A1,A,Tue,09:00-18:00,Seat_C1,27
1,A1,A,Wed,09:00-18:00,Seat_B1,27
2,A1,A,Thu,09:00-18:00,Seat_B1,27
3,A2,A,Mon,09:00-18:01,Seat_A1,21
4,A2,A,Wed,09:00-18:01,Seat_A1,21
...,...,...,...,...,...,...
76,E2,E,Tue,20:30-05:30,Seat_B1,4
77,E2,E,Thu,20:30-05:30,Seat_C3,4
78,E3,E,Tue,22:00-07:00,Seat_C1,19
79,E3,E,Thu,22:00-07:00,Seat_A1,19


In [8]:
try:
    schedule_df.to_csv("seat_schedule_summary.csv")
    expanded_df.to_csv("seat_schedule_expanded.csv")
    print("\nSaved CSVs: seat_schedule_summary.csv, seat_schedule_expanded.csv")
except Exception:
    pass


Saved CSVs: seat_schedule_summary.csv, seat_schedule_expanded.csv
